# TCJA Permanency Act
To run this notebook, you need the following:
1. Fork the repository https://github.com/OpenSourceEcon/TCJAperm and clone it to your local machine.
2. Navigate to the `/TCJAperm/` directory in your terminal and create the `tcjaperm-dev` conda environment by typing: `conda env create -f environment.yml`
3. Open the `TCJA_perm_taxcalc.ipynb` notebook and set the kernel in the notebook to the `tcjaperm-dev` conda environment.
4. Execute the cells in the `TCJA_perm_taxcalc.ipynb` notebook.
    * You will need to adjust the file path to the `puf.csv` file in code cell 2 and the path to the `TCJA_perm.json` file in code cell 3.
    * If you do not have the IRS Public Use File, then you will need to use the CPS file included with Tax-Calculator. That code is commented out in cell 2.

In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import taxcalc as tc
import matplotlib.pyplot as plt

## 1. Create TCJA Permanency Reform

### 1.1. Get the overall cost estimate change

Create the baseline calculator object.

In [2]:
puf_path = (
    "/Users/richardevans/Docs/Economics/OSE/Tax-Calculator/taxcalc/puf.csv"
)
recs = tc.Records(data=puf_path)

In [ ]:
# If you need to use the CPS data file included with Tax-Calculator, comment
# out the previous lines in this cell and uncomment the following line.
# recs = tc.Records.cps_constructor()
pol_ref1 = tc.Policy()
calc_base_ref1 = tc.Calculator(policy=pol_ref1, records=recs)
CYR = 2024
calc_base_ref1.advance_to_year(CYR)
calc_base_ref1.calc_all()
itax_rev_base_ref1 = calc_base_ref1.weighted_total('iitax')

Create the reform `json` object and corresponding calculator object.

In [ ]:
reform_json_tcjaperm = {
    "ALD_AlimonyPaid_hc": {"2026": 1.0},
    "ALD_AlimonyReceived_hc": {"2026": 0.0},
    "ALD_DomesticProduction_hc": {"2026": 1.0},
    "ALD_BusinessLosses_c": {"2026": [270000.0, 540000.0, 270000.0, 270000.0, 540000.0]},
    "II_em": {"2026": 0},
    "II_em_ps": {"2026": [9e99, 9e99, 9e99, 9e99, 9e99]},
    "STD": {"2026": [12950.0, 19400.0, 12950.0, 19400.0, 25900.0]},
    "ID_AllTaxes_c": {"2026": [10000.0, 10000.0, 5000.0, 10000.0, 10000.0]},
    "ID_Charity_crt_all": {"2026": 0.6},
    "ID_Casualty_hc": {"2026": 1.0},
    "ID_Miscellaneous_hc": {"2026": 1.0},
    "ID_ps": {"2026": [9e99, 9e99, 9e99, 9e99, 9e99]},
    "ID_prt": {"2026": 0.0},
    "ID_crt": {"2026": 1.0},
    "II_rt2": {"2026": 0.12},
    "II_rt3": {"2026": 0.22},
    "II_rt4": {"2026": 0.24},
    "II_rt5": {"2026": 0.32},
    "II_rt7": {"2026": 0.37},
    "II_brk1": {"2026": [10275.0, 20550.0, 10275.0, 14650.0, 20550.0]},
    "II_brk2": {"2026": [41775.0, 83550.0, 41775.0, 55900.0, 83550.0]},
    "II_brk3": {"2026": [89075.0, 178150.0, 89075.0, 89050.0, 178150.0]},
    "II_brk4": {"2026": [170050.0, 340100.0, 170050.0, 170050.0, 340100.0]},
    "II_brk5": {"2026": [215950.0, 431900.0, 215950.0, 215950.0, 431900.0]},
    "II_brk6": {"2026": [539900.0, 647850.0, 323925.0, 539900.0, 647850.0]},
    "PT_rt2": {"2026": 0.12},
    "PT_rt3": {"2026": 0.22},
    "PT_rt4": {"2026": 0.24},
    "PT_rt5": {"2026": 0.32},
    "PT_rt7": {"2026": 0.37},
    "PT_brk1": {"2026": [10275.0, 20550.0, 10275.0, 14650.0, 20550.0]},
    "PT_brk2": {"2026": [41775.0, 83550.0, 41775.0, 55900.0, 83550.0]},
    "PT_brk3": {"2026": [89075.0, 178150.0, 89075.0, 89050.0, 178150.0]},
    "PT_brk4": {"2026": [170050.0, 340100.0, 170050.0, 170050.0, 340100.0]},
    "PT_brk5": {"2026": [215950.0, 431900.0, 215950.0, 215950.0, 431900.0]},
    "PT_brk6": {"2026": [539900.0, 647850.0, 323925.0, 539900.0, 647850.0]},
    "PT_qbid_rt": {"2026": 0.2},
    "PT_qbid_taxinc_thd": {"2026": [170050.0, 340100.0, 170050.0, 170050.0, 340100.0]},
    "PT_qbid_taxinc_gap": {"2026": [50000.0, 100000.0, 50000.0, 50000.0, 100000.0]},
    "PT_qbid_w2_wages_rt": {"2026": 0.5},
    "PT_qbid_alt_w2_wages_rt": {"2026": 0.25},
    "PT_qbid_alt_property_rt": {"2026": 0.025},
    "AMT_em": {"2026": [75900.0, 118100.0, 59050.0, 75900.0, 118100.0]},
    "AMT_em_ps": {"2026": [539900.0, 1079800.0, 539900.0, 539900.0, 1079800.0]},
    "AMT_em_pe": {"2026": 776100.0},
    "CTC_c": {"2026": 2000.0},
    "CTC_ps": {"2026": [200000.0, 400000.0, 200000.0, 200000.0, 400000.0]},
    "ACTC_c": {"2026": 1600.0},
    "ODC_c": {"2026": 500.0},
    "ACTC_Income_thd": {"2026": 2500.0}
}

In [ ]:
# reform_json_path = (
#     "/Users/richardevans/Docs/Economics/OSE/TCJAperm/json/TCJA_perm.json"
# )
# params_ref1 = pol_ref1.read_json_reform(reform_json_path)
params_ref1 = pol_ref1.read_json_reform(reform_json_tcjaperm)
pol_ref1.implement_reform(params_ref1)
calc_ref_ref1 = tc.Calculator(policy=pol_ref1, records=recs)

In [ ]:
# Compute annual revenue for each year in a 10 year budget window
# Do this for with and without the TCJA permanency reform
rev_base_ref1 = {'IIT': {}}
rev_ref_ref1 = {'IIT': {}}
for t in range(2024, 2034):
    calc_base_ref1.advance_to_year(t)
    calc_base_ref1.calc_all()
    calc_ref_ref1.advance_to_year(t)
    calc_ref_ref1.calc_all()
    rev_base_ref1['IIT'][t] = calc_base_ref1.weighted_total('iitax')
    rev_ref_ref1['IIT'][t] = calc_ref_ref1.weighted_total('iitax')

In [ ]:
# make table of revenue estimates
rev_base_ref1_df = pd.DataFrame.from_dict(rev_base_ref1).T
rev_ref_ref1_df = pd.DataFrame.from_dict(rev_ref_ref1).T
diff_ref1_df = (rev_ref_ref1_df - rev_base_ref1_df) * 1e-9  # in $ billions
pd.options.display.float_format = '${:.3f}'.format
diff_ref1_df['2024-2033'] = diff_ref1_df.sum(axis=1)
# print(diff_ref1_df.to_latex())
diff_ref1_df

### 1.2. Get the section-specific decomposition of overall cost estimate change

Create baseline scenario.

In [ ]:
pol_ref1_dec = tc.Policy()
calc_base_ref1_dec = tc.Calculator(policy=pol_ref1_dec, records=recs)
CYR = 2024
calc_base_ref1_dec.advance_to_year(CYR)
calc_base_ref1_dec.calc_all()
itax_rev_base_ref1_dec = calc_base_ref1_dec.weighted_total('iitax')

Create a sequential analysis of the reform of each part of the TCJA permanency reform.

In [3]:
sec_ref1_list = [
    (
        "Sec 1: (II_rt*, II_brk*, PT_rt*, PT_brk*, PT_qbid*)",
        {
            "II_rt2": {"2026": 0.12},
            "II_rt3": {"2026": 0.22},
            "II_rt4": {"2026": 0.24},
            "II_rt5": {"2026": 0.32},
            "II_rt7": {"2026": 0.37},
            "II_brk1": {"2026": [10275.0, 20550.0, 10275.0, 14650.0, 20550.0]},
            "II_brk2": {"2026": [41775.0, 83550.0, 41775.0, 55900.0, 83550.0]},
            "II_brk3": {"2026": [89075.0, 178150.0, 89075.0, 89050.0, 178150.0]},
            "II_brk4": {"2026": [170050.0, 340100.0, 170050.0, 170050.0, 340100.0]},
            "II_brk5": {"2026": [215950.0, 431900.0, 215950.0, 215950.0, 431900.0]},
            "II_brk6": {"2026": [539900.0, 647850.0, 323925.0, 539900.0, 647850.0]},
            "PT_rt2": {"2026": 0.12},
            "PT_rt3": {"2026": 0.22},
            "PT_rt4": {"2026": 0.24},
            "PT_rt5": {"2026": 0.32},
            "PT_rt7": {"2026": 0.37},
            "PT_brk1": {"2026": [10275.0, 20550.0, 10275.0, 14650.0, 20550.0]},
            "PT_brk2": {"2026": [41775.0, 83550.0, 41775.0, 55900.0, 83550.0]},
            "PT_brk3": {"2026": [89075.0, 178150.0, 89075.0, 89050.0, 178150.0]},
            "PT_brk4": {"2026": [170050.0, 340100.0, 170050.0, 170050.0, 340100.0]},
            "PT_brk5": {"2026": [215950.0, 431900.0, 215950.0, 215950.0, 431900.0]},
            "PT_brk6": {"2026": [539900.0, 647850.0, 323925.0, 539900.0, 647850.0]},
            "PT_qbid_rt": {"2026": 0.2},
            "PT_qbid_taxinc_thd": {"2026": [170050.0, 340100.0, 170050.0, 170050.0, 340100.0]},
            "PT_qbid_taxinc_gap": {"2026": [50000.0, 100000.0, 50000.0, 50000.0, 100000.0]},
            "PT_qbid_w2_wages_rt": {"2026": 0.5},
            "PT_qbid_alt_w2_wages_rt": {"2026": 0.25},
            "PT_qbid_alt_property_rt": {"2026": 0.025}
        }
    ),
    (
        "Sec 2: (STD, ID_*)",
        {
            "STD": {"2026": [12950.0, 19400.0, 12950.0, 19400.0, 25900.0]},
            "ID_AllTaxes_c": {"2026": [10000.0, 10000.0, 5000.0, 10000.0, 10000.0]},
            "ID_Charity_crt_all": {"2026": 0.6},
            "ID_Casualty_hc": {"2026": 1.0},
            "ID_Miscellaneous_hc": {"2026": 1.0},
            "ID_ps": {"2026": [9e99, 9e99, 9e99, 9e99, 9e99]},
            "ID_prt": {"2026": 0.0},
            "ID_crt": {"2026": 1.0},
        }
    ),
    (
        "Sec 3: (CTC_*, ACTC_*, ODC_c)",
        {
            "CTC_c": {"2026": 2000.0},
            "CTC_ps": {"2026": [200000.0, 400000.0, 200000.0, 200000.0, 400000.0]},
            "ACTC_c": {"2026": 1600.0},
            "ODC_c": {"2026": 500.0},
            "ACTC_Income_thd": {"2026": 2500.0}
        }
    ),
    (
        "Sec 4: (AMT_*)",
        {
            "AMT_em": {"2026": [75900.0, 118100.0, 59050.0, 75900.0, 118100.0]},
            "AMT_em_ps": {"2026": [539900.0, 1079800.0, 539900.0, 539900.0, 1079800.0]},
            "AMT_em_pe": {"2026": 776100.0},
        }
    ),
    (
        "Sec 5: (II_em*)",
        {
            "II_em": {"2026": 0},
            "II_em_ps": {"2026": [9e99, 9e99, 9e99, 9e99, 9e99]}
        }
    ),
    (
        "Sec 6: (ALD_*)",
        {
            "ALD_AlimonyPaid_hc": {"2026": 1.0},
            "ALD_AlimonyReceived_hc": {"2026": 0.0},
            "ALD_DomesticProduction_hc": {"2026": 1.0}
        }
    )
]

Create the difference between the Sec 1 reform and the baseline

In [ ]:
params_ref1_sec1 = pol_ref1_dec.read_json_reform(sec_ref1_list[0][1])
pol_ref1_dec.implement_reform(params_ref1_sec1)
calc_ref_ref1_sec1 = tc.Calculator(policy=pol_ref1_dec, records=recs)

In [ ]:
rev_base_ref1_sec1 = {'IIT': {}}
rev_ref_ref1_sec1 = {'IIT': {}}
for t in range(2024, 2034):
    calc_base_ref1_dec.advance_to_year(t)
    calc_base_ref1_dec.calc_all()
    calc_ref_ref1_sec1.advance_to_year(t)
    calc_ref_ref1_sec1.calc_all()
    rev_base_ref1_sec1['IIT'][t] = calc_base_ref1_dec.weighted_total('iitax')
    rev_ref_ref1_sec1['IIT'][t] = calc_ref_ref1_sec1.weighted_total('iitax')

In [ ]:
rev_base_ref1_sec1_df = pd.DataFrame.from_dict(rev_base_ref1_sec1).T
rev_ref_ref1_sec1_df = pd.DataFrame.from_dict(rev_ref_ref1_sec1).T
diff_ref1_sec1_df = (rev_ref_ref1_sec1_df - rev_base_ref1_sec1_df) * 1e-9  # in $ billions
pd.options.display.float_format = '${:.3f}'.format
diff_ref1_sec1_df['2024-2033'] = diff_ref1_sec1_df.sum(axis=1)
# print(diff_ref1_sec1_df.to_latex())
diff_ref1_sec1_df

## Write a loop that compares each section to the original baseline

In [7]:
# Create blank pandas dataframe with columns ["Provisions", "2024", "2025",..., "2033", "2024-2033"]
diff_df = pd.DataFrame(
    columns=[
        "Provisions", "2024", "2025", "2026", "2027", "2028", "2029", "2030",
        "2031", "2032", "2033", "2024-2033"
    ]
)

for i,tpl in enumerate(sec_ref1_list):
    print("")
    print("Simulating " + tpl[0])
    pol = tc.Policy()
    calc_base = tc.Calculator(policy=pol, records=recs)
    CYR = 2024
    calc_base.advance_to_year(CYR)
    calc_base.calc_all()

    params_ref = pol.read_json_reform(tpl[1])
    pol.implement_reform(params_ref)
    calc_ref = tc.Calculator(policy=pol, records=recs)

    rev_base = {'IIT': {}}
    rev_ref = {'IIT': {}}
    for t in range(2024, 2034):
        calc_base.advance_to_year(t)
        calc_base.calc_all()
        calc_ref.advance_to_year(t)
        calc_ref.calc_all()
        rev_base['IIT'][t] = calc_base.weighted_total('iitax')
        rev_ref['IIT'][t] = calc_ref.weighted_total('iitax')

    rev_base_df = pd.DataFrame.from_dict(rev_base).T
    rev_ref_df = pd.DataFrame.from_dict(rev_ref).T
    diff_sec_df = (rev_ref_df - rev_base_df) * 1e-9  # in $ billions
    pd.options.display.float_format = '${:.3f}'.format
    diff_sec_df['2024-2033'] = diff_sec_df.sum(axis=1)
    print(diff_sec_df)
    diff_df.loc[i, "Provisions"] = tpl[0]
    numeric_columns = [
        "2024", "2025", "2026", "2027", "2028", "2029", "2030", "2031", "2032",
        "2033", "2024-2033"
    ]
    for col in numeric_columns:
        diff_df.loc[i, col] = diff_sec_df[col].values[0]

diff_df


Simulating Sec 1: (II_rt*, II_brk*, PT_rt*, PT_brk*, PT_qbid*)
      2024   2025      2026      2027      2028      2029      2030      2031  \
IIT $0.000 $0.000 $-143.121 $-149.832 $-156.407 $-162.981 $-169.666 $-176.234   

         2032      2033  2024-2033  
IIT $-181.282 $-189.343 $-1328.866  


KeyError: '2024'

## 2. Create Simple Reform

Create a new baseline for simple reform.

In [ ]:
pol_simp = tc.Policy()
calc_base_simp = tc.Calculator(policy=pol_simp, records=recs)
CYR = 2024
calc_base_simp.advance_to_year(CYR)
calc_base_simp.calc_all()
itax_rev_base_simp = calc_base_simp.weighted_total('iitax')

Create a reform json and corresponding calculator that is simple and straightforward.

In [ ]:
reform_json_simp = {
    "II_rt1": {
        "2024": 0.08,
        "2025": 0.08,
        "2026": 0.08
    },
    "II_rt2": {
        "2024": 0.10,
        "2025": 0.10,
        "2026": 0.10
    },
    "II_rt3": {
        "2024": 0.20,
        "2025": 0.20,
        "2026": 0.20
    }
}

In [ ]:
params_simp = pol_simp.read_json_reform(reform_json_simp)
pol_simp.implement_reform(params_simp)
calc_ref_simp = tc.Calculator(policy=pol_simp, records=recs)

In [ ]:
rev_base_simp = {'IIT': {}, 'Payroll': {}}
rev_ref_simp = {'IIT': {}, 'Payroll': {}}
for t in range(2024, 2034):
    calc_base_simp.advance_to_year(t)
    calc_base_simp.calc_all()
    calc_ref_simp.advance_to_year(t)
    calc_ref_simp.calc_all()
    rev_base_simp['IIT'][t] = calc_base_simp.weighted_total('iitax')
    rev_base_simp['Payroll'][t] = calc_base_simp.weighted_total('payrolltax')
    rev_ref_simp['IIT'][t] = calc_ref_simp.weighted_total('iitax')
    rev_ref_simp['Payroll'][t] = calc_ref_simp.weighted_total('payrolltax')

In [ ]:
rev_base_simp_df = pd.DataFrame.from_dict(rev_base_simp).T
rev_ref_simp_df = pd.DataFrame.from_dict(rev_ref_simp).T
diff_simp_df = (rev_ref_simp_df - rev_base_simp_df) * 1e-9  # in $ billions
pd.options.display.float_format = '${:.3f}'.format
diff_simp_df['2024-2033'] = diff_simp_df.sum(axis=1)
# print(diff_simp_df.to_latex())
diff_simp_df

# Distributional results

See below + table by Cody

In [ ]:
# Restart calculation to get back to first year
pol = taxcalc.policy.Policy()
tc_base = taxcalc.calculator.Calculator(policy=pol, records=rec)
pol.implement_reform(ref)
tc_arpa = taxcalc.calculator.Calculator(policy=pol, records=rec)
tc_base.advance_to_year(2021)
tc_arpa.advance_to_year(2021)
tc_base.calc_all()
tc_arpa.calc_all()

# get data for plots
mars = 'ALL'
mtr_wrt_full_compen = True
record_variables = ['s006', 'XTOT']
if mars != 'ALL':
    record_variables.append('MARS')
record_variables.append('expanded_income')
record_variables.append('e00200')
vdf = tc_base.dataframe(record_variables)
# create 'tax1' and 'tax2' columns given specified atr_measure
vdf['tax1'] = tc_base.array('iitax')
vdf['tax2'] = tc_arpa.array('iitax')
# compute mtrs for mtr plot data
(mtr1_ptax, mtr1_itax, mtr1_combined) = tc_base.mtr(
    variable_str='e00200p', wrt_full_compensation=mtr_wrt_full_compen)
(mtr2_ptax, mtr2_itax, mtr2_combined) = tc_arpa.mtr(
    variable_str='e00200p', wrt_full_compensation=mtr_wrt_full_compen)
vdf['mtr1'] = mtr1_itax
vdf['mtr2'] = mtr2_itax

mtr_data = taxcalc.calculator.mtr_graph_data(vdf,
                              year=2021,
                              mars=mars,
                              mtr_measure='itax',
                              alt_e00200p_text='This text',
                              mtr_wrt_full_compen=True,
                              income_measure='expanded_income',
                              pop_quantiles=True,
                              dollar_weighting=False)
atr_data = taxcalc.calculator.atr_graph_data(vdf,
                              year=2021,
                              mars=mars,
                              atr_measure='itax',
                              pop_quantiles=True)

In [ ]:
plt.plot(mtr_data['lines'].index, mtr_data['lines'].base.values, label='Pre-ARPA')
plt.plot(mtr_data['lines'].index, mtr_data['lines'].reform.values, label='Post-ARPA')
plt.xlabel('Income Percentile')
plt.ylabel('Marginal Tax Rate on Wage Income')
plt.legend(loc=9, bbox_to_anchor=(0.5, -0.15), ncol=2)
plt.savefig('arpa_mtr_plot.png', bbox_inches='tight')

In [ ]:
plt.plot(atr_data['lines'].index, atr_data['lines'].base.values, label='Pre-ARPA')
plt.plot(atr_data['lines'].index, atr_data['lines'].reform.values, label='Post-ARPA')
plt.xlabel('Income Percentile')
plt.ylabel('Average Tax Rate')
plt.legend(loc=9, bbox_to_anchor=(0.5, -0.15), ncol=2)
plt.savefig('arpa_atr_plot.png', bbox_inches='tight')

# Tax benefits by family size

In [ ]:
df = tc_base.dataframe(['nu18', 'XTOT', 'iitax', 'c00100', 's006', 'aftertax_income', 'expanded_income'])
df['ARPA_iitax'] = tc_arpa.array('iitax')
df['tax_change'] = df['ARPA_iitax'] - df['iitax']
df['aftertax_income_ARPA'] = tc_arpa.array('aftertax_income')
df.groupby('nu18')[['s006', 'XTOT']].sum()

In [ ]:
mdf.weighted_mean(df, 'nu18', 's006')

In [ ]:
decile_bounds = np.arange(0, 1.1, 0.1)
deciles = mdf.weighted_quantile(df, 'c00100', 's006', decile_bounds)
quintile_bounds = np.arange(0, 1.2, 0.2)
quintiles = mdf.weighted_quantile(df, 'c00100', 's006', quintile_bounds)

In [ ]:
df_group = mdf.weighted_mean(df, 'tax_change', w='s006', groupby=['nu18'])
df_group.plot.bar()

In [ ]:
from cycler import cycler
from cycler import cycler
N = 7
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.Blues(np.linspace(0.2,1,N)))


df['quintiles'] = pd.cut(df['expanded_income'], bins=quintiles, labels=np.arange(1,6).tolist())
df_group = pd.DataFrame(mdf.weighted_mean(df, 'tax_change', w='s006', groupby=['nu18', 'quintiles']))
# df_group.plot.bar()
df_group.reset_index(inplace=True)
df_group.rename(columns={0: 'Average Change in Taxes', 'nu18': 'Number of Children', 'quintiles': 'Income Quintiles'}, inplace=True)
df_group['Number of Children'] = df_group['Number of Children'].astype('int64')
ax1 = df_group.pivot("Income Quintiles", "Number of Children", "Average Change in Taxes").plot(kind='bar', alpha=1.0)
plt.ylabel('Mean Tax Change')
plt.xticks(rotation = 0)
plt.legend(title='Number of Children', loc=9, bbox_to_anchor=(0.5, -0.15), ncol=3)
plt.savefig('bar_plot_by_kids.png', bbox_inches='tight')

In [ ]:
df.drop(df[df['expanded_income'] < 0].index, inplace=True)
percentile_bounds = np.arange(0, 1.01, 0.01)
percentiles = mdf.weighted_quantile(df, 'expanded_income', 's006', percentile_bounds)
df['Percentile'] = pd.cut(df['expanded_income'], bins=percentiles, labels=np.arange(1,101).tolist())
df_group = pd.DataFrame(mdf.weighted_mean(
    df, ['aftertax_income', 'aftertax_income_ARPA'], w='s006', groupby='Percentile'))
df_group.reset_index(inplace=True)
df_group.plot(kind='line', alpha=1.0)
plt.ylim([0, 50000])
# plt.ylabel('Mean Tax Change')
# plt.xticks(rotation = 0)
# plt.legend(title='Number of Children', loc=9, bbox_to_anchor=(0.5, -0.15), ncol=3)
# plt.savefig('bar_plot_by_kids.png', bbox_inches='tight')


In [ ]:
df_group

In [ ]:
decile_bounds = np.arange(0, 1.1, 0.1)

In [ ]:
pct_bounds = np.array([0, 0.2, 0.4, 0.6, 0.8, 0.9, 0.99, 1.0])
percentiles = mdf.weighted_quantile(df, 'cexpanded_income', 's006', pct_bounds)
df['Percentile'] = pd.cut(df['expanded_income'], bins=percentiles)
df_group = pd.DataFrame(mdf.weighted_mean(
    df, ['aftertax_income', 'aftertax_income_ARPA'], w='s006', groupby='Percentile'))
df_group.reset_index(inplace=True)
df_group.plot(kind='line', alpha=1.0)
# plt.ylim([0, 50000])